<a href="https://colab.research.google.com/github/Edgar-La/hacking-civico/blob/master/proyecto/proyecto-hacking-civico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Titulo:__
En este documento encontras lo siguiente:

* Descripcion:

* Selección y obtención de los datos

* Exploración de los datos

* Visualización de los datos

* Conclusión 

* Referencias


## __Descripcion.__
Información de los registros de la deuda pública contraída vigente (acreedor, acreditado, monto, garantías, tasas, comisiones, plazos, amortizaciones, destino, etc.).
Auditoría Superior del Estado de Guanajuato

## __Seleccion y obtencion de datos.__

In [1]:
#Importar paquetes de Python
import pandas as pd
import numpy as np
import altair as alt
import statistics

In [2]:
#Para descargar los datos
!wget http://www.aseg.gob.mx/descarga/deuda_publica.csv

--2020-10-11 03:14:31--  http://www.aseg.gob.mx/descarga/deuda_publica.csv
Resolving www.aseg.gob.mx (www.aseg.gob.mx)... 201.147.245.170
Connecting to www.aseg.gob.mx (www.aseg.gob.mx)|201.147.245.170|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32904 (32K) [application/octet-stream]
Saving to: ‘deuda_publica.csv’

deuda_publica.csv   100%[===================>]  32.13K  95.0KB/s    in 0.3s    

2020-10-11 03:14:32 (95.0 KB/s) - ‘deuda_publica.csv’ saved [32904/32904]



In [160]:
#Convertir datos en DataFrame
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'deuda_publica.csv'
# Utilizamos la función de Pandas para cargar un archivo CSV
Datos = pd.read_csv(nombre_archivo, encoding = 'latin-1')

In [161]:
# Limpieza del nombre de las columnas, remover espacios y '.'
Datos.columns = Datos.columns.str.replace(' ', '_')
Datos.columns = Datos.columns.str.replace('.', '')
Datos.columns = Datos.columns.str.replace('(', '')
Datos.columns = Datos.columns.str.replace(')', '')

#Limpieza de lo que contienen ciertas columnas, como '$', ',', y también pasamos los NaN -> 0 con fillna(0) indicando la columna
for t in Datos.Monto_original_contratado_en_pesos:
    Datos.Monto_original_contratado_en_pesos = Datos.Monto_original_contratado_en_pesos.str.replace('$', '')
    Datos.Monto_original_contratado_en_pesos = Datos.Monto_original_contratado_en_pesos.str.replace(',', '') 
Datos['Monto_original_contratado_en_pesos'] = Datos['Monto_original_contratado_en_pesos'].fillna(0)                         ### Nuevo comando  ###

for t in Datos.Monto_dispuesto_en_pesos:
    Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace('$', '')
    Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace(',', '')
    #Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace('NaN', '0')
    Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace('Total', '0')
Datos['Monto_dispuesto_en_pesos'] = Datos['Monto_dispuesto_en_pesos'].fillna(0)

for t in Datos.Saldo_en_pesos	:
    Datos.Saldo_en_pesos = Datos.Saldo_en_pesos.str.replace('$', '')
    Datos.Saldo_en_pesos = Datos.Saldo_en_pesos.str.replace(',', '')
Datos['Saldo_en_pesos'] = Datos['Saldo_en_pesos'].fillna(0)

#Limpieza minima a dato extra excrito
for t in Datos.Plazo_máximo_meses:
  Datos.Plazo_máximo_meses = Datos.Plazo_máximo_meses.str.replace(' meses', '')

Datos['Fecha_de_Vencimiento'] = Datos['Fecha_de_Vencimiento'].fillna(0)  

Datos   #Muestra cabecera

,Ente_Acreditado,No_de_Crédito,No_de_inscripción_SHCP,Registro_Estatal,Ambito,Institución_Acreedora,Tipo_de_Obligación,Clase_del_titulo,Monto_original_contratado_en_pesos,Monto_dispuesto_en_pesos,Saldo_en_pesos,Tasa,Fecha_de_Contratación,Plazo_máximo_meses,Fecha_de_Vencimiento,Garantía,Destino
0,Gobierno del Estado de Guanajuato,NaN,167/2002,124/02,Estatal,"Banco del Bajío, S.A.",Contrato de Apertura de Crédito Simple,Pesos,233000000.00,232851337.75,15597340.75,TIIE +1.00,09/12/2002,180,27/11/2017,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
1,Gobierno del Estado de Guanajuato,1466.0,169/02,125/02,Estatal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,Pesos,289037000.00,289037000.00,52301933.78,FOAEM Nominal + 1.50,18/12/2002,240,10/03/2023,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
2,Gobierno del Estado de Guanajuato,1467.0,169/02,125/02,Estatal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,UDIS,289037000.00,289037000.00,84325399.36,FOAEM Real + 1.50,18/12/2002,240,10/03/2023,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
3,Tarimoro,4470.0,276/2004,140/04,Municipal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,Pesos,3132842.97,3132842.97,671324.13,TIIE * 1.120,11/11/2004,180,23/12/2019,Participaciones Federales,Se destinará única y exclusivamente en los sig...
4,Gobierno del Estado de Guanajuato,5768.0,234/2005,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,1184414400.00,97109245.00,CETES 182 + 1.34,20/05/2005,216,01/02/2022,Participaciones Federales,Obra pública
5,Gobierno del Estado de Guanajuato,6342.0,234/2005,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,1184414400.00,88311746.67,CETES 182 + 1.33,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
6,Gobierno del Estado de Guanajuato,6938.0,234/2005,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,1184414400.00,36064550.65,CETES 182 + 1.08,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
7,Gobierno del Estado de Guanajuato,7061.0,234/2006,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,1184414400.00,106248939.08,CETES 182 + 1.11,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
8,Gobierno del Estado de Guanajuato,7848.0,234/2006,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,1184414400.00,84958370.54,CETES 182 + 1.22,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
9,Jerécuaro,7073.0,352/2007,168/07,Municipal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,4382506.00,4382506.00,191225.21,TIIE + 1.81,07/12/2007,120,02/01/2018,Participaciones Federales,Financiar los siguientes conceptos: \nTramo 1....


In [162]:
#Podemos notar que los valores que nos interesan siguen siendo tipo object
Datos.dtypes

Ente_Acreditado                        object
No_de_Crédito                         float64
No_de_inscripción_SHCP                 object
Registro_Estatal                       object
Ambito                                 object
Institución_Acreedora                  object
Tipo_de_Obligación                     object
Clase_del_titulo                       object
Monto_original_contratado_en_pesos     object
Monto_dispuesto_en_pesos               object
Saldo_en_pesos                         object
Tasa                                   object
Fecha_de_Contratación                  object
Plazo_máximo_meses                     object
Fecha_de_Vencimiento                   object
Garantía                               object
Destino                                object
dtype: object

In [163]:
#Convertimos las columnas que nos interesan a numeric con pd.to_numeric()
Datos['Monto_original_contratado_en_pesos'] = pd.to_numeric(Datos['Monto_original_contratado_en_pesos'])
Datos['Monto_dispuesto_en_pesos'] = pd.to_numeric(Datos['Monto_dispuesto_en_pesos'])
Datos['Saldo_en_pesos'] = pd.to_numeric(Datos['Saldo_en_pesos'])
Datos['Plazo_máximo_meses'] = pd.to_numeric(Datos['Plazo_máximo_meses'])
Datos['Fecha_de_Vencimiento'] = pd.to_datetime(Datos['Fecha_de_Vencimiento'])

Datos.dtypes

Ente_Acreditado                               object
No_de_Crédito                                float64
No_de_inscripción_SHCP                        object
Registro_Estatal                              object
Ambito                                        object
Institución_Acreedora                         object
Tipo_de_Obligación                            object
Clase_del_titulo                              object
Monto_original_contratado_en_pesos           float64
Monto_dispuesto_en_pesos                     float64
Saldo_en_pesos                               float64
Tasa                                          object
Fecha_de_Contratación                         object
Plazo_máximo_meses                           float64
Fecha_de_Vencimiento                  datetime64[ns]
Garantía                                      object
Destino                                       object
dtype: object

## __Exploracion de los datos.__

In [164]:
#Datos.groupby('Ente_Acreditado').sum()
Datos.groupby('Institución_Acreedora').Ente_Acreditado.count()      #Filtrado por institucion acredora
#Datos.query('Ente_Acreditado == "Guanajuato"').head()

Institución_Acreedora
BBVA Bancomer                                             6
Banamex                                                   7
Banco del Bajío, S.A.                                    19
Banobras S.N.C                                           19
Banorte                                                   1
Concesionaria del Acueducto El Zapotillo S.A. de C.V.     1
HSBC                                                      2
Scotiabank                                                1
Name: Ente_Acreditado, dtype: int64

In [24]:
#Filtro para Guanajuato
#Datos.query('Ente_Acreditado == "Gobierno del Estado de Guanajuato"').Monto_original_contratado_en_pesos.value_counts()

In [72]:
Datos_L = Datos.groupby('Ente_Acreditado').Monto_original_contratado_en_pesos.describe();
#Datos_L = Datos_L.convert_dtypes;
Datos_L

,count,mean,std,min,25%,50%,75%,max
Ente_Acreditado,,,,,,,,
Abasolo,1.0,1.300000e+07,NaN,1.300000e+07,1.300000e+07,1.300000e+07,1.300000e+07,1.300000e+07
Apaseo el Grande,1.0,1.980342e+07,NaN,1.980342e+07,1.980342e+07,1.980342e+07,1.980342e+07,1.980342e+07
Celaya,1.0,3.500000e+08,NaN,3.500000e+08,3.500000e+08,3.500000e+08,3.500000e+08,3.500000e+08
"Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.",1.0,3.310000e+09,NaN,3.310000e+09,3.310000e+09,3.310000e+09,3.310000e+09,3.310000e+09
"Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.",1.0,4.300000e+07,NaN,4.300000e+07,4.300000e+07,4.300000e+07,4.300000e+07,4.300000e+07
Comonfort,1.0,1.986088e+07,NaN,1.986088e+07,1.986088e+07,1.986088e+07,1.986088e+07,1.986088e+07
Cuerámaro,2.0,7.750000e+06,6.363961e+05,7.300000e+06,7.525000e+06,7.750000e+06,7.975000e+06,8.200000e+06
Doctor Mora,1.0,7.493849e+06,NaN,7.493849e+06,7.493849e+06,7.493849e+06,7.493849e+06,7.493849e+06
Gobierno del Estado de Guanajuato,21.0,8.824193e+08,5.997558e+08,2.330000e+08,5.000000e+08,6.289892e+08,1.184414e+09,2.152000e+09


In [73]:
Datos_L2 = Datos.groupby('Ente_Acreditado').Monto_dispuesto_en_pesos.describe();
Datos_L2

,count,mean,std,min,25%,50%,75%,max
Ente_Acreditado,,,,,,,,
Abasolo,1.0,1.300000e+07,NaN,1.300000e+07,1.300000e+07,1.300000e+07,1.300000e+07,1.300000e+07
Apaseo el Grande,1.0,1.255940e+07,NaN,1.255940e+07,1.255940e+07,1.255940e+07,1.255940e+07,1.255940e+07
Celaya,1.0,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.",1.0,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.",1.0,3.705161e+07,NaN,3.705161e+07,3.705161e+07,3.705161e+07,3.705161e+07,3.705161e+07
Comonfort,1.0,1.986088e+07,NaN,1.986088e+07,1.986088e+07,1.986088e+07,1.986088e+07,1.986088e+07
Cuerámaro,2.0,7.750000e+06,6.363961e+05,7.300000e+06,7.525000e+06,7.750000e+06,7.975000e+06,8.200000e+06
Doctor Mora,1.0,7.493849e+06,NaN,7.493849e+06,7.493849e+06,7.493849e+06,7.493849e+06,7.493849e+06
Gobierno del Estado de Guanajuato,21.0,7.050132e+08,5.008764e+08,0.000000e+00,2.890370e+08,5.000000e+08,1.184414e+09,1.664477e+09


In [74]:
Datos_L3 = Datos.groupby('Ente_Acreditado').Saldo_en_pesos.describe();
Datos_L3

,count,mean,std,min,25%,50%,75%,max
Ente_Acreditado,,,,,,,,
Abasolo,1.0,1.747899e+06,NaN,1.747899e+06,1.747899e+06,1.747899e+06,1.747899e+06,1.747899e+06
Apaseo el Grande,1.0,7.644852e+06,NaN,7.644852e+06,7.644852e+06,7.644852e+06,7.644852e+06,7.644852e+06
Celaya,1.0,7.955556e+07,NaN,7.955556e+07,7.955556e+07,7.955556e+07,7.955556e+07,7.955556e+07
"Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.",1.0,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.",1.0,2.169534e+07,NaN,2.169534e+07,2.169534e+07,2.169534e+07,2.169534e+07,2.169534e+07
Comonfort,1.0,8.879630e+06,NaN,8.879630e+06,8.879630e+06,8.879630e+06,8.879630e+06,8.879630e+06
Cuerámaro,2.0,5.125232e+06,1.964671e+06,3.736000e+06,4.430616e+06,5.125232e+06,5.819849e+06,6.514465e+06
Doctor Mora,1.0,1.663398e+06,NaN,1.663398e+06,1.663398e+06,1.663398e+06,1.663398e+06,1.663398e+06
Gobierno del Estado de Guanajuato,21.0,2.850091e+08,4.597897e+08,1.559734e+07,8.432540e+07,1.683599e+08,2.254000e+08,2.126972e+09


In [75]:
Datos_M = Datos.groupby('Ente_Acreditado').Monto_original_contratado_en_pesos.value_counts()
#Datos_M = pd.DataFrame(data = Datos_M);
Datos_M

Ente_Acreditado                                                                                  Monto_original_contratado_en_pesos
Abasolo                                                                                          1.300000e+07                          1
Apaseo el Grande                                                                                 1.980342e+07                          1
Celaya                                                                                           3.500000e+08                          1
Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.  3.310000e+09                          1
Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.                             4.300000e+07                          1
Comonfort                                                                                        1.986088e+07                          1
Cuerámaro                                     

In [ ]:
#Datos_K = Datos.query('Ente_Acreditado == "Guanajuato"').Monto_dispuesto_en_pesos.value_counts()
#Datos_K

In [78]:
Datos_L5 = Datos.groupby('Ente_Acreditado').Plazo_máximo_meses.describe();
#Datos_L5 = Datos.groupby('Ente_Acreditado').Plazo_máximo_meses.value_counts()
Datos_L5

,count,mean,std,min,25%,50%,75%,max
Ente_Acreditado,,,,,,,,
Abasolo,1.0,120.0,NaN,120.0,120.0,120.0,120.0,120.0
Apaseo el Grande,1.0,120.0,NaN,120.0,120.0,120.0,120.0,120.0
Celaya,1.0,180.0,NaN,180.0,180.0,180.0,180.0,180.0
"Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.",1.0,300.0,NaN,300.0,300.0,300.0,300.0,300.0
"Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.",1.0,180.0,NaN,180.0,180.0,180.0,180.0,180.0
Comonfort,1.0,180.0,NaN,180.0,180.0,180.0,180.0,180.0
Cuerámaro,2.0,180.0,0.000000,180.0,180.0,180.0,180.0,180.0
Doctor Mora,1.0,120.0,NaN,120.0,120.0,120.0,120.0,120.0
Gobierno del Estado de Guanajuato,21.0,160.0,59.194594,60.0,120.0,180.0,216.0,240.0


In [85]:
Datos_L6 = Datos.groupby('Ente_Acreditado').Fecha_de_Vencimiento.describe();
#Datos_L5 = Datos.groupby('Ente_Acreditado').Fecha_de_Vencimiento.value_counts()
Datos_L6

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:679: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  result = self.apply(lambda x: x.describe(**kwargs))
/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:679: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  result = self.apply(lambda x: x.describe(**kwargs))
/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:679: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is

,count,unique,top,freq,first,last
Ente_Acreditado,,,,,,
Abasolo,1,1,2018-04-30,1,2018-04-30,2018-04-30
Apaseo el Grande,1,1,2021-09-08,1,2021-09-08,2021-09-08
Celaya,1,1,2031-10-31,1,2031-10-31,2031-10-31
"Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.",1,1,1970-01-01,1,1970-01-01,1970-01-01
"Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.",1,1,2026-12-14,1,2026-12-14,2026-12-14
Comonfort,1,1,2029-02-27,1,2029-02-27,2029-02-27
Cuerámaro,2,2,2024-02-08,1,2024-02-08,2028-06-11
Doctor Mora,1,1,2019-09-24,1,2019-09-24,2019-09-24
Gobierno del Estado de Guanajuato,21,16,2022-01-02,5,2017-11-27,2031-03-23


## __Visualizacion de los datos.__

In [135]:
Grafico_2 = alt.Chart(Datos).mark_bar().encode(
    x = 'Ente_Acreditado',
    y = 'Monto_original_contratado_en_pesos',
    color = 'Ente_Acreditado'
)
Grafico_2

alt.Chart(...)

In [176]:
#¿Qué Institución Acreedora es la más utilizada?
a = alt.Chart(Datos).mark_bar().encode(
    y = alt.Y('Institución_Acreedora', title = 'Institucion Acreedora'),
    x = alt.X('count(Institución_Acreedora)', title = 'Cantidad de créditos'),
    color = alt.Color('Institución_Acreedora', legend = None),
)

b = alt.Chart(Datos).mark_text(size = 15, dx=-9).encode(
    y = alt.Y('Institución_Acreedora'),
    x = alt.X('count(Institución_Acreedora)'),
    text = alt.Text('count(Institución_Acreedora)'),
)

c = alt.layer(a,b).properties(title = 'Cantidad de créditos por Institución Acreedora')
c

alt.LayerChart(...)

In [205]:
#¿Cuál es la media de los Plazos máximos en meses por Ente Acreditado?
d = alt.Chart(Datos).mark_bar(fillOpacity = 0.70).encode(
    x = alt.X('Ente_Acreditado', title = 'Ente Acreditado'),
    y = alt.Y('Plazo_máximo_meses', title = 'Plazo máximo (meses)'),
    color = alt.Color('Ente_Acreditado', legend = None),
)
e = alt.Chart(Datos).mark_text(size = 16, dy=10).encode(
    x = alt.X('Ente_Acreditado'),
    y = alt.Y('Plazo_máximo_meses'),
    text = alt.Text('max(Plazo_máximo_meses)'),
)
f = alt.layer(d, e).properties(
    title = 'Duración de los Plazos Acreditados por Entidad',
    height = 250,
    width = 800,)
f

alt.LayerChart(...)

In [122]:
#¿Cualos es la media de los Plazos máximos en mese por Ente Acreditado?
Uno = alt.Chart(Datos).mark_bar().encode(
    x = 'Fecha_de_Vencimiento:T',
    y = 'Ente_Acreditado:O',
    color = 'Ente_Acreditado')
Uno
#'''
#Dos = alt.Chart(Datos).mark_text(dx=-40).encode(
#    x = alt.X('Fecha_de_Vencimiento:T'),
#    y = alt.Y('Ente_Acreditado:O'),
#    text = alt.Text('max(Fecha_de_Vencimiento):T'),
#)
#Uno + Dos'''

alt.Chart(...)

## __Conclusion.__

## __Referencias.__